<a href="https://colab.research.google.com/github/Francisss3/AAI612_Francis/blob/main/Notbook5_3_lab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


# AAI612: Deep Learning & its Applications

*Notebook 5.3: KerasTuner*

<a href="https://colab.research.google.com/github/harmanani/AAI612/blob/main/Week5/Notebook5.3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Ensure that KerasTuner is installed:

In [1]:
!pip install keras-tuner -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.1/129.1 kB 3.0 MB/s eta 0:00:00


Import KerasTuner and TensorFlow:

In [2]:
import keras_tuner
from tensorflow import keras
from tensorflow.keras import layers

`KerasTuner` lets you replace hard-coded hyperparameter values, such as units=32, with a range of possible choices, such as `Int(name="units", min_value=16, max_value=64, step=16)`. This set of choices in a given model is called the search space of the hyperparameter tuning process.  To specify a search space, define a model-building function (see the next listing).  It takes an hp argument, from which you can sample hyperparameter ranges, and it returns a compiled Keras model.

In [3]:
def build_model(hp):
    units = hp.Int(name="units", min_value=16, max_value=64, step=16)
    model = keras.Sequential([
        layers.Dense(units, activation="relu"),
        layers.Dense(10, activation="softmax")
    ])
    optimizer = hp.Choice(name="optimizer", values=["rmsprop", "adam"])
    model.compile(
        optimizer=optimizer,
        loss="sparse_categorical_crossentropy",
        metrics=["accuracy"])
    return model

**A KerasTuner `HyperModel`**

We can also adopt a more modular and configurable approach to model-building by subclassing the HyperModel class and define a build method, as follows:

In [4]:
import keras_tuner as kt

class SimpleMLP(kt.HyperModel):
    def __init__(self, num_classes):
        self.num_classes = num_classes

    def build(self, hp):
        units = hp.Int(name="units", min_value=16, max_value=64, step=16)
        model = keras.Sequential([
            layers.Dense(units, activation="relu"),
            layers.Dense(self.num_classes, activation="softmax")
        ])
        optimizer = hp.Choice(name="optimizer", values=["rmsprop", "adam"])
        model.compile(
            optimizer=optimizer,
            loss="sparse_categorical_crossentropy",
            metrics=["accuracy"])
        return model

hypermodel = SimpleMLP(num_classes=10)

The next step is to define a “tuner.” Schematically, you can think of a tuner as a for loop that will repeatedly
- Pick a set of hyperparameter values
- Call the model-building function with these values to create a model
- Train the model and record its metrics

KerasTuner has several built-in tuners available—RandomSearch, BayesianOptimization, and Hyperband. Let’s try BayesianOptimization, a tuner that attempts to make smart predictions for which new hyperparameter values are likely to perform best given the outcomes of previous choices:

In [5]:
tuner = kt.BayesianOptimization(
    build_model,
    objective="val_accuracy",
    max_trials=100,
    executions_per_trial=2,
    directory="mnist_kt_test",
    overwrite=True,
)

Display an overview of the search space via `search_space_summary()`

In [6]:
tuner.search_space_summary()

Search space summary
Default search space size: 2
units (Int)
{'default': None, 'conditions': [], 'min_value': 16, 'max_value': 64, 'step': 16, 'sampling': 'linear'}
optimizer (Choice)
{'default': 'rmsprop', 'conditions': [], 'values': ['rmsprop', 'adam'], 'ordered': False}


In [7]:
(x_train, y_train), (x_test, y_test) = keras.datasets.mnist.load_data()
x_train = x_train.reshape((-1, 28 * 28)).astype("float32") / 255
x_test = x_test.reshape((-1, 28 * 28)).astype("float32") / 255
x_train_full = x_train[:]
y_train_full = y_train[:]
num_val_samples = 10000
x_train, x_val = x_train[:-num_val_samples], x_train[-num_val_samples:]
y_train, y_val = y_train[:-num_val_samples], y_train[-num_val_samples:]
callbacks = [
    keras.callbacks.EarlyStopping(monitor="val_loss", patience=5),
]
tuner.search(
    x_train, y_train,
    batch_size=128,
    epochs=100,
    validation_data=(x_val, y_val),
    callbacks=callbacks,
    verbose=2,
)

Trial 36 Complete [00h 00m 43s]
val_accuracy: 0.9740500152111053

Best val_accuracy So Far: 0.9771499931812286
Total elapsed time: 00h 29m 12s

Search: Running Trial #37

Value             |Best Value So Far |Hyperparameter
64                |64                |units
rmsprop           |rmsprop           |optimizer

Epoch 1/100
391/391 - 3s - 8ms/step - accuracy: 0.8870 - loss: 0.4303 - val_accuracy: 0.9331 - val_loss: 0.2434
Epoch 2/100
391/391 - 1s - 3ms/step - accuracy: 0.9364 - loss: 0.2261 - val_accuracy: 0.9465 - val_loss: 0.1915
Epoch 3/100
391/391 - 1s - 3ms/step - accuracy: 0.9503 - loss: 0.1756 - val_accuracy: 0.9591 - val_loss: 0.1522
Epoch 4/100
391/391 - 1s - 3ms/step - accuracy: 0.9581 - loss: 0.1456 - val_accuracy: 0.9595 - val_loss: 0.1443
Epoch 5/100
391/391 - 1s - 2ms/step - accuracy: 0.9642 - loss: 0.1246 - val_accuracy: 0.9651 - val_loss: 0.1289
Epoch 6/100
391/391 - 1s - 2ms/step - accuracy: 0.9692 - loss: 0.1077 - val_accuracy: 0.9670 - val_loss: 0.1180
Epoch 7/100

KeyboardInterrupt: 

In [ ]:
import tensorflow as tf
from tensorflow import keras
from kerastuner.tuners import RandomSearch

# Load and preprocess the MNIST dataset
(x_train, y_train), (x_test, y_test) = keras.datasets.mnist.load_data()
x_train = x_train.reshape((-1, 28 * 28)).astype("float32") / 255
x_test = x_test.reshape((-1, 28 * 28)).astype("float32") / 255

# Create a validation set
num_val_samples = 10000
x_train, x_val = x_train[:-num_val_samples], x_train[-num_val_samples:]
y_train, y_val = y_train[:-num_val_samples], y_train[-num_val_samples:]

# Define EarlyStopping callback
callbacks = [
    keras.callbacks.EarlyStopping(monitor="val_loss", patience=5),
]

# Define the model builder function
def build_model(hp):
    model = keras.Sequential([
        keras.layers.Dense(
            units=hp.Int("units", min_value=32, max_value=512, step=32),
            activation="relu",
            input_shape=(28 * 28,)
        ),
        keras.layers.Dense(10, activation="softmax")
    ])
    model.compile(
        optimizer=keras.optimizers.Adam(
            hp.Choice("learning_rate", values=[0.001, 0.0001])
        ),
        loss="sparse_categorical_crossentropy",
        metrics=["accuracy"]
    )
    return model

# Define the tuner
tuner = RandomSearch(
    build_model,
    objective="val_loss",
    max_trials=10,
    executions_per_trial=1,
    directory="my_tuner_results",
    project_name="mnist_tuning"
)

# Run hyperparameter search
tuner.search(
    x_train, y_train,
    batch_size=128,
    epochs=100,
    validation_data=(x_val, y_val),
    callbacks=callbacks,
    verbose=2,
)


Trial 5 Complete [00h 00m 20s]
val_loss: 0.07117322087287903

Best val_loss So Far: 0.06949639320373535
Total elapsed time: 00h 02m 44s

Search: Running Trial #6

Value             |Best Value So Far |Hyperparameter
192               |384               |units
0.001             |0.001             |learning_rate

Epoch 1/100


## Querying the best hyperparameter configurations

Once the search is complete, you can query the best hyperparameter configurations, which you can use to create high-performing models that you can then retrain:

In [1]:
top_n = 4
best_hps = tuner.get_best_hyperparameters(top_n)

NameError: name 'tuner' is not defined

Before we can train on the full training data, though, there’s one last parameter we need to settle: the optimal number of epochs to train for. Typically, you’ll want to train the new models for longer than you did during the search: using an aggressive patience value in the EarlyStopping callback saves time during the search, but it may lead to under-fit models. Just use the validation set to find the best epoch:

In [ ]:
def get_best_epoch(hp):
    model = build_model(hp)
    callbacks=[
        keras.callbacks.EarlyStopping(
            monitor="val_loss", mode="min", patience=10)
    ]
    history = model.fit(
        x_train, y_train,
        validation_data=(x_val, y_val),
        epochs=100,
        batch_size=128,
        callbacks=callbacks)
    val_loss_per_epoch = history.history["val_loss"]
    best_epoch = val_loss_per_epoch.index(min(val_loss_per_epoch)) + 1
    print(f"Best epoch: {best_epoch}")
    return best_epoch

Finally, train on the full dataset for just a bit longer than this epoch count, since
you’re training on more data; 20% more in this case:

In [ ]:
def get_best_trained_model(hp):
    best_epoch = get_best_epoch(hp)
    model = build_model(hp)
    model.fit(
        x_train_full, y_train_full,
        batch_size=128, epochs=int(best_epoch * 1.2))
    return model

best_models = []
for hp in best_hps:
    model = get_best_trained_model(hp)
    model.evaluate(x_test, y_test)
    best_models.append(model)

In [ ]:
best_models = tuner.get_best_models(top_n)